# What-if scenario evaluation for carbon sequestration

This notebook uses the [COMET-Farm API](https://gitlab.com/comet-api/api-docs/-/tree/master/) to derive carbon sequestration information for agricultural fields.

To install the required packages, see [this README file](README.md).

To perform the analysis, the API requires some JSON files with information about the fields. They are:

 * [`baseline.json`](./baseline.json): Contains information about agriculture activities performed during previous seasons. The previous seasons' information should be repeated from the year 2000 to the (current calendar year - 1).
 * [`historical.json`](./historical.json): Contains details of crop history.
 * [`scenario.json`](./scenario.json): Contains information about agriculture activities to be performed in the current calendar year. This will be used by the COMET-Farm API to generate the analysis results.
 * [`boundary.json`](./boundary.json) - Contains details about the agriculture farmland.

Examples for all the files mentioned above are available alongside this notebook, and the names of the files themselves are clickable, allowing exploration of the values.

In code cells below, we will use these examples to evaluate a scenario.

In [ ]:
import json
from vibe_core.data import CarbonScenario
from vibe_core.data.carbon import Baseline, CBoundary, Historical, Scenario

historical_file = "./historical.json"
with open(historical_file) as json_file:
    c_historcical = Historical(**json.load(json_file))

baseline_file = "./baseline.json"
with open(baseline_file) as json_file:
    c_baseline = Baseline(**json.load(json_file))

scenario_file = "./scenario.json"
with open(scenario_file) as json_file:
    c_scenario = Scenario(**json.load(json_file))

boundary_file = "./boundary.json"
with open(boundary_file) as json_file:
    c_boundary = CBoundary(**json.load(json_file))

inputs = CarbonScenario(
    boundary=c_boundary,
    scenario=c_scenario,
    baseline=c_baseline,
    historical=c_historcical
)

## Details about the output of the COMET-Farm API

1. All greenhouse gas models in the COMET-Farm platform (Example: DayCent, rice
   methane, residue burning, liming, urea fertilizer, etc.) are run against the
   baseline scenario and then against each conservation scenario, on each unique
   combination of soil map units found within each parcel or point for each model
   run.

2.	Aggregated Baseline and Scenario results totals for all models are also
    returned, named "Baseline" and "Scenario".

An example output follows:

```json
{
    "@name": "scenario: 21/07/2022 10:34:05",
    "Carbon": {
        "SoilCarbon": "-1679.9",
        "BiomassBurningCarbon": "0",
        "SoilCarbonStock2000": "5511.312",
        "SoilCarbonStockBegin": "5753.6314",
        "SoilCarbonStockEnd": "5759.8725"
    },
    "CO2": {
        "LimingCO2": "0",
        "UreaFertilizationCO2": "8.5587",
        "DrainedOrganicSoilsCO2": "0"
    },
    "N2O": {
        "SoilN2O": "536.1286",
        "SoilN2O_Direct": "451.9349",
        "SoilN2O_Indirect_Volatilization": "84.1937",
        "SoilN2O_Indirect_Leaching": "0",
        "WetlandRiceCultivationN2O": "0",
        "BiomassBurningN2O": "0",
        "DrainedOrganicSoilsN2O": "0"
    },
    "CH4": {
        "SoilCH4": "0",
        "WetlandRiceCultivationCH4": "0",
        "BiomassBurningCH4": "0"
    }
}
```

## Pre-requisites to run this notebook

1. Sign up with https://comet-farm.com/. The email registered there will be used when there are error messages, or a request fails.
2. Sign up with https://dashboard.ngrok.com/ to allow the TerraVibes carbon endpoint to be accessible by COMET-Farm API's webhooks.
   1. Navigate to "Getting Started"/"Your Auth token" and copy the Auth token
   2. Update copied auth token in variable "NGROK_AUTH_TOKEN" in next cell

In [ ]:
NGROK_AUTH_TOKEN = ""
COMET_REGISTERED_EMAIL = ""

In [ ]:
import os
from datetime import datetime, timezone

from vibe_core.datamodel import RunStatus
from vibe_core.client import FarmvibesAiClient, get_default_vibe_client, get_local_service_url

### On running this workflow

*Note*: Running this workflow will expose an endpoint within the FarmVibes.AI worker container publicly. This connection will exist for as long as the workflow is running, and will be closed as soon as the workflow completes.

This is a required step for receiving results from the COMET-Farm API. Failure to do so, will prevent the workflow from completing.

In [ ]:
client: FarmvibesAiClient = get_default_vibe_client()

start_date = datetime(year=2021, month=2, day=1, tzinfo=timezone.utc)
end_date = datetime(year=2021, month=2, day=11, tzinfo=timezone.utc)

parameters = {
  "ngrok_token": NGROK_AUTH_TOKEN,
  "comet_support_email": COMET_REGISTERED_EMAIL
}

run = client.run("farm_ai/carbon_local/carbon_whatif", name="test_carbon_seq", input_data=inputs, parameters=parameters)

try:
  run.block_until_complete(30)
except RuntimeError as e:
  print(run)

Unless the workflow completed under 30s, which requires a response from the COMET-Farm API, the output of the workflow will not be available yet.

Use the cells below to inspect the running workflow.

Depending on the COMET-Farm API resources availability, workflows will take longer to complete.

If the workflow does not complete withing 2 (two) hours and the workflow is still running, use the COMET-Farm support at appnrel@colostate.edu.

On failure to execute the COMET-Farm request, error information will be sent to the email registered with COMET.

In [ ]:
client.describe_run(run.id)

In [ ]:
client.get_run_by_id(run.id)